In [ ]:
import os
import torch
import torch.utils.data import DataLodaer

In [ ]:
if not os.path.exists("Ultrasound"):
    !git clone https://github.com/z872845991/Ultrasound.git
%cd Ultrasound
!ls

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
x_transforms = transforms.Compose([
    transforms.Resize((512,512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
# mask只需要转换为tensor
y_transforms = transforms.Compose([
    transforms.Resize((512, 512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
   ])